In [70]:
import pandas as pd
import numpy as np

In [71]:
binding_db = pd.read_csv(filepath_or_buffer='./raw_data/BindingDB_Covid-19.tsv', sep='\t')
alt_binding_db = pd.read_csv(filepath_or_buffer='./raw_data/alt.tsv', sep='\t')


useful_col_names = ['BindingDB Ligand Name', 'Ligand SMILES', 'IC50 (nM)']

binding_db = binding_db[useful_col_names]
alt_binding_db = alt_binding_db[useful_col_names]

binding_df = pd.concat([binding_db, alt_binding_db], axis=0)

binding_df.columns = ['ligand_name', 'smiles', 'IC50_nM']

print(binding_df.shape)

binding_df.head()

(13851, 3)


,ligand_name,smiles,IC50_nM
0,AG7088 analogue 2a::CHEMBL20636::N-[(5-methyli...,CCOC(=O)\C=C\[C@H](C[C@@H]1CCNC1=O)NC(=O)[C@H]...,NaN
1,AG7088 analogue 2d::CHEMBL277716::N-[(benzylox...,CCOC(=O)\C=C\[C@H](C[C@@H]1CCNC1=O)NC(=O)[C@H]...,NaN
2,"N-[(benzyloxy)carbonyl]-L-valyl-N1-((1S,2E)-4-...",CCOC(=O)\C=C\[C@H](C[C@@H]1CCNC1=O)NC(=O)[C@H]...,NaN
3,N-[(benzyloxy)carbonyl]-O-(tert-butyl)-L-threo...,CCOC(=O)\C=C\[C@H](C[C@@H]1CCNC1=O)NC(=O)[C@H]...,NaN
4,N-[(benzyloxy)carbonyl]-O-(tert-butyl)-L-threo...,C[C@H](OC(C)(C)C)[C@H](NC(=O)OCc1ccccc1)C(=O)N...,NaN


In [72]:
def convert_ic50(value):
    # Удаляем символы ">" или "<" из строки
    value = str(value).replace('<', '').replace('>', '').strip()
    
    try:
        # Пробуем преобразовать в число с плавающей запятой
        return float(value)
    except ValueError:
        # Если преобразование невозможно, возвращаем NaN
        return np.nan

In [73]:
binding_df = binding_df[(~binding_df['IC50_nM'].isnull()) & (binding_df['IC50_nM'] != 'IC50 (nM)')]

binding_df.drop_duplicates(subset='smiles', inplace=True)

binding_df['IC50_nM_parsed'] = binding_df['IC50_nM'].apply(convert_ic50)

binding_df.drop(columns='IC50_nM', axis=1, inplace=True)

binding_df.sort_values(by='IC50_nM_parsed', inplace=True, ascending=False)

binding_df_trainset = binding_df.head(510)
    
    
binding_df_trainset['SI'] = 0.00

binding_df_trainset.head()

C:\Users\roxasmeei\AppData\Local\Temp\ipykernel_16896\1581351015.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binding_df_trainset['SI'] = 0.00


,ligand_name,smiles,IC50_nM_parsed,SI
5911,CHEBI:5801::Hydroxychloroquine::acs.jmedchem.1...,CCN(CCO)CCCC(C)Nc1ccnc2cc(Cl)ccc12,3584000.0,0.0
1239,"WO2006061714, P38.5::cmdc.202100576, 10f",COc1cccc2[nH]c(cc12)C(=O)N[C@@H](CC(C)C)C(=O)N...,2300000.0,0.0
5909,CHEMBL1929014::acs.jmedchem.1c00409_ST.815,CC(C)C[C@H](NC(=O)[C@@H](NC(=O)[C@H](C)NC(=O)[...,2000000.0,0.0
2,CHEMBL4633785,OB1Oc2c(ccc(F)c2C(O)=O)[C@H]2C[C@@H]12,1000000.0,0.0
65,"2-[(4-hydroxy-5,6-dimethylpyrimidin-2-yl)sulfa...",Cc1nc(SCC(=O)Nc2ccc(cc2)[N+]([O-])=O)[n-]c(=[O...,1000000.0,0.0


In [75]:
binding_df_trainset.to_csv('./processed_data/non_inhibitors_db.csv', index=False)